## TODOs:
* Need to apply cache to all subsets of list

In [1]:
from fastai2.text.all import *
from pigboat.text.all import *
# from fastai2.basics import *

In [2]:
# _olditem = TfmdLists.__getitem__
# @patch
# def __getitem__(self:TfmdLists, idx):
#     set_trace()
#     _olditem(self, idx)

In [3]:
source = untar_data(URLs.IMDB_SAMPLE)

In [4]:
df = pd.read_csv(source/'texts.csv')
df.head(1)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False


In [5]:
df['text2'] = df['text']

In [6]:
ABSTAIN,POS,NEG = 'abstain','positive','negative'

In [7]:
labeller = Labeller(ABSTAIN)
col_reader,tokenizer,numericalize = mk_transform(ColReader('text2')),Tokenizer.from_df('text'),Numericalize

In [8]:
@labeller(col_reader)
def liked(x): return POS if 'liked' in x else None
@labeller(col_reader)
def hated(x): return NEG if 'hated' in x else None

In [9]:
splits = ColSplitter()(df)

In [10]:
vocab = [ABSTAIN,POS,NEG]

In [11]:
#TODO: How to not pass ABSTAIN?
# Adds option to cache results, should also fix #1
tasks = TfmdLists(df, [col_reader,tokenizer,numericalize,CategorizeTaskLabels(vocab)], splits=splits)
tasks.cache(attrgetter('labels'))

AttributeError: cache

In [ ]:
true_lbls = TfmdLists(df, [ColReader('label'), Categorize(vocab=vocab)], splits=splits)

In [ ]:
dset_train = Datasets(tasks).train
dset_valid = Datasets(tls=[tasks,true_lbls]).valid

In [ ]:
dls = DataLoaders.from_dsets(dset_train, dset_valid, drop_last=False)

In [ ]:
def acc(inp,targ): return (inp==targ).float().mean()

In [ ]:
learn = Learner(dls, MajorityLabelVoter(), loss_func=lambda *_: tensor(0.), metrics=acc)

In [ ]:
learn.validate()